编译器（Compiler）：一次性将源代码（如C++）转换为机器码，生成独立可执行文件。
优点：执行效率高（直接运行机器码）。
缺点：调试困难，跨平台需重新编译。
示例：GCC、Clang。
解释器（Interpreter：逐行翻译并执行代码（如Python）。
优点：跨平台灵活，易于调试。
缺点：执行速度慢（需实时翻译）。
示例：CPython、Ruby解释器。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.0582,  0.1935]], grad_fn=<AddmmBackward>)

In [2]:
net = torch.jit.script(net)
net(x)

tensor([[-0.0582,  0.1935]], grad_fn=<AddBackward0>)

In [3]:
#@save
class Benchmark:
    """用于测量运行时间"""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [4]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

无torchscript: 0.1780 sec
有torchscript: 0.2201 sec


### 序列化
编译模型的好处之一是我们可以将模型及其参数序列化（保存）到磁盘。这允许这些训练好的模型部署到其他设备上，并且还能方便地使用其他前端编程语言

## 12.2. 异步计算

In [6]:
import os
import subprocess
import numpy
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
# GPU计算热身
device = d2l.try_gpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.5255 sec
torch: 0.0021 sec


In [8]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.2017 sec


PyTorch有一个用于与用户直接交互的前端（例如通过Python），还有一个由系统用来执行计算的后端。

In [9]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='cuda:0')

## 12.4. 硬件

In [10]:
import torch
from d2l import torch as d2l

In [ ]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])